In [1]:
import pandas
import string
import nltk
import keras
import sklearn
import time
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy
import tensorflow as tf
import numpy as np
from time import time
from nltk.corpus import stopwords
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.callbacks import TensorBoard
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
import keras.preprocessing.text

from tensorflow.keras.utils import to_categorical

In [3]:
# load dataset
dataframe = pandas.read_csv("dataset.csv", header=0)
dataframe.drop_duplicates(inplace = True)
data= dataframe
print(dataframe.head())


     page                                            comment  scale
0  Page_1  tento areal je v katastrofalnom stave aspon 10...     -2
1  Page_1  skoda len ze ponukaju aj stale tie iste a rovn...     -2
2  Page_1  Mali by ste dat tabulky "VSTUP DO BRATISLAVY N...     -2
3  Page_1  Za to, ako dlho sa to stavia, tam mala byt pos...     -2
4  Page_1  A kde je preboha nejaká zeleň ?????? 😞😞😕☹️☹️☹️...     -2


In [5]:
print(data.head())

                                             comment  scale
0  tento areal je v katastrofalnom stave aspon 10...     -2
1  skoda len ze ponukaju aj stale tie iste a rovn...     -2
2  Mali by ste dat tabulky "VSTUP DO BRATISLAVY N...     -2
3  Za to, ako dlho sa to stavia, tam mala byt pos...     -2
4  A kde je preboha nejaká zeleň ?????? 😞😞😕☹️☹️☹️...     -2


In [4]:
data.drop('page', inplace=True, axis=1)

In [31]:
#Create scale column using Score
data=data.replace({'scale' : { -2:0,-1:1,0:2,1:3,2:4 }})

In [32]:
train_size = int(len(data) * .8)

print(int(len(data['comment'])))
print(train_size)

1587
1269


In [33]:
texts= data['comment']
tags = data['scale']


train_posts = data['comment'][:train_size]
train_tags = data['scale'][:train_size]



test_posts = data['comment'][train_size:]
test_tags =  data['scale'][train_size:]

In [34]:
tokenizer = Tokenizer(num_words=None,lower=False)
tokenizer.fit_on_texts(texts)

x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')


In [35]:
x_train.shape

(1269, 11109)

In [36]:
x_test.shape

(318, 11109)

In [37]:
encoder = LabelEncoder()
encoder.fit(tags)
tagst=encoder.fit_transform(tags)

num_classes = int((len(set(tagst))))
print((len(set(tagst))))

y_train = encoder.fit_transform(train_tags)
y_test = encoder.fit_transform(test_tags)



5


In [38]:
num_classes

5

In [39]:
y_train= to_categorical(y_train,num_classes)
y_test = to_categorical(y_test, num_classes)


num_labels = int(len(y_train.shape))
vocab_size = len(tokenizer.word_index) + 1

max_words=vocab_size


In [40]:
import keras.backend as K
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

from keras.metrics import Precision , Recall , Accuracy , TruePositives , TrueNegatives , FalsePositives , FalseNegatives


In [41]:
# Build the model
model = Sequential()
model.add(Dense(1024, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [42]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','Recall','Precision', f1_metric,'TruePositives','TrueNegatives','FalsePositives','FalseNegatives'])


In [43]:
batch_size = 100
epochs = 50

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


model.save('my_model.h5')


Epoch 1/50
12/12 [==============================] - 2s 90ms/step - loss: 1.4486 - categorical_accuracy: 0.3406 - recall: 0.7496 - precision: 0.2949 - f1_metric: 0.4250 - true_positives: 856.0000 - true_negatives: 2521.0000 - false_positives: 2047.0000 - false_negatives: 286.0000 - val_loss: 2.5748 - val_categorical_accuracy: 0.0000e+00 - val_recall: 0.0079 - val_precision: 0.0027 - val_f1_metric: 0.0025 - val_true_positives: 1.0000 - val_true_negatives: 136.0000 - val_false_positives: 372.0000 - val_false_negatives: 126.0000
Epoch 2/50
12/12 [==============================] - 0s 41ms/step - loss: 0.4979 - categorical_accuracy: 0.9168 - recall: 0.9877 - precision: 0.3806 - f1_metric: 0.5509 - true_positives: 1128.0000 - true_negatives: 2732.0000 - false_positives: 1836.0000 - false_negatives: 14.0000 - val_loss: 2.3771 - val_categorical_accuracy: 0.0236 - val_recall: 0.3071 - val_precision: 0.1021 - val_f1_metric: 0.1768 - val_true_positives: 39.0000 - val_true_negatives: 165.0000 - val

Epoch 17/50
12/12 [==============================] - 0s 41ms/step - loss: 0.0061 - categorical_accuracy: 0.9991 - recall: 1.0000 - precision: 0.8088 - f1_metric: 0.8955 - true_positives: 1142.0000 - true_negatives: 4298.0000 - false_positives: 270.0000 - false_negatives: 0.0000e+00 - val_loss: 3.1659 - val_categorical_accuracy: 0.0866 - val_recall: 0.4646 - val_precision: 0.1634 - val_f1_metric: 0.2782 - val_true_positives: 59.0000 - val_true_negatives: 206.0000 - val_false_positives: 302.0000 - val_false_negatives: 68.0000
Epoch 18/50
12/12 [==============================] - 0s 42ms/step - loss: 0.0057 - categorical_accuracy: 0.9991 - recall: 1.0000 - precision: 0.8263 - f1_metric: 0.9061 - true_positives: 1142.0000 - true_negatives: 4328.0000 - false_positives: 240.0000 - false_negatives: 0.0000e+00 - val_loss: 3.1804 - val_categorical_accuracy: 0.0945 - val_recall: 0.4646 - val_precision: 0.1639 - val_f1_metric: 0.2785 - val_true_positives: 59.0000 - val_true_negatives: 207.0000 - v

12/12 [==============================] - 0s 40ms/step - loss: 0.0029 - categorical_accuracy: 0.9991 - recall: 1.0000 - precision: 0.8612 - f1_metric: 0.9276 - true_positives: 1142.0000 - true_negatives: 4384.0000 - false_positives: 184.0000 - false_negatives: 0.0000e+00 - val_loss: 3.2908 - val_categorical_accuracy: 0.1181 - val_recall: 0.5039 - val_precision: 0.1823 - val_f1_metric: 0.3001 - val_true_positives: 64.0000 - val_true_negatives: 221.0000 - val_false_positives: 287.0000 - val_false_negatives: 63.0000
Epoch 33/50
12/12 [==============================] - 0s 40ms/step - loss: 0.0030 - categorical_accuracy: 0.9991 - recall: 1.0000 - precision: 0.8586 - f1_metric: 0.9247 - true_positives: 1142.0000 - true_negatives: 4380.0000 - false_positives: 188.0000 - false_negatives: 0.0000e+00 - val_loss: 3.2902 - val_categorical_accuracy: 0.1260 - val_recall: 0.5197 - val_precision: 0.1880 - val_f1_metric: 0.3132 - val_true_positives: 66.0000 - val_true_negatives: 223.0000 - val_false_pos

12/12 [==============================] - 0s 40ms/step - loss: 0.0023 - categorical_accuracy: 0.9991 - recall: 1.0000 - precision: 0.8554 - f1_metric: 0.9215 - true_positives: 1142.0000 - true_negatives: 4375.0000 - false_positives: 193.0000 - false_negatives: 0.0000e+00 - val_loss: 3.3004 - val_categorical_accuracy: 0.1339 - val_recall: 0.5433 - val_precision: 0.2023 - val_f1_metric: 0.3246 - val_true_positives: 69.0000 - val_true_negatives: 236.0000 - val_false_positives: 272.0000 - val_false_negatives: 58.0000
Epoch 48/50
12/12 [==============================] - 1s 43ms/step - loss: 0.0022 - categorical_accuracy: 0.9991 - recall: 1.0000 - precision: 0.8718 - f1_metric: 0.9317 - true_positives: 1142.0000 - true_negatives: 4400.0000 - false_positives: 168.0000 - false_negatives: 0.0000e+00 - val_loss: 3.2621 - val_categorical_accuracy: 0.1339 - val_recall: 0.5512 - val_precision: 0.2059 - val_f1_metric: 0.3277 - val_true_positives: 70.0000 - val_true_negatives: 238.0000 - val_false_pos

In [44]:
from tensorflow.keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [45]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [46]:
#model = keras.models.load_model('my_model.h1')
Evaluation_valus = model.evaluate(x_test,y_test,verbose=0)
print("Loss" , 'categorical_accuracy','Recall','Precision','f1_metric','TruePositives','TrueNegatives','FalsePositives','FalseNegatives')

print(Evaluation_valus)


Loss categorical_accuracy Recall Precision f1_metric TruePositives TrueNegatives FalsePositives FalseNegatives
[2.191162109375, 0.26100629568099976, 0.6886792182922363, 0.2576470673084259, 0.3742395341396332, 219.0, 641.0, 631.0, 99.0]


In [47]:
#Predictions
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing import sequence
import numpy as np
import pickle

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
print("Loading saved tokenizer...")

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("my_model.h5")
print("Loading saved model...")

def predict_response(response):
    '''
    0 - very negative
    1 - somewhat negative
    2 - neutral
    3 - somewhat positive
    4 - very positive
    '''
    print(f'You said: \n{response}\n')
    response = tokenizer.texts_to_sequences(response)
    response = sequence.pad_sequences(response, maxlen=11109)
    pred = np.argmax(model.predict(response))
    if pred == 0:
        return 'Very Negative'
    elif pred == 1:
        return 'Somewhat Negative'
    elif pred == 2:
        return 'Nuetral'
    elif pred == 3:
        return 'Somewhat Positive'
    elif pred == 4:
        return 'Very Positive'   

Loading saved tokenizer...
Loading saved model...


In [58]:
user_input = input()
response = predict_response(response=[user_input])
print(response)


Život bez chorob.Laska bez omedzení.
You said: 
['Život bez chorob.Laska bez omedzení.']

Somewhat Negative
